In [8]:
!pip install json
!pip install azureml-core
!pip install azureml-sdk



ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


     ---------------------------------------- 3.3/3.3 MB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 55.5/55.5 kB ? eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 26.2 MB/s eta 0:00:00
     ------------------------------------- 207.1/207.1 kB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 24.6 MB/s eta 0:00:00
     -------------------------------------- 142.5/142.5 kB 8.8 MB/s eta 0:00:00
     ---------------------------------------- 40.8/40.8 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 128.7/128.7 kB ? eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB ? eta 0:00:00
     ---------------------------------------- 85.4/85.4 kB ? eta 0:00:00
     ---------------------------------------- 43.7/43.7 kB ? eta 0:00:00
     ---------------------------------------- 46.4/46.4 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 114.7/114.7 kB 6.5 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


     ---------------------------------------- 8.6/8.6 MB 24.0 MB/s eta 0:00:00
     ---------------------------------------- 137.7/137.7 kB ? eta 0:00:00
     ---------------------------------------- 252.4/252.4 kB ? eta 0:00:00
     --------------------------------------- 25.5/25.5 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 69.7/69.7 kB ? eta 0:00:00
     ------------------------------------- 313.8/313.8 kB 20.2 MB/s eta 0:00:00
     ------------------------------------- 245.7/245.7 kB 14.7 MB/s eta 0:00:00
     --------------------------------------- 18.3/18.3 MB 24.2 MB/s eta 0:00:00
     -------------------------------------- 189.2/189.2 kB 5.8 MB/s eta 0:00:00
     ------------------------------------- 890.9/890.9 kB 19.1 MB/s eta 0:00:00
     ---------------------------------------- 55.1/55.1 kB 3.0 MB/s eta 0:00:00
  Created wheel for fusepy: filename

In [10]:
#GETTING MY ID:
import json

id = open('id.json', 'r')
mi = json.load(id)
azure_id = mi['azure_id']

from azureml.core import Workspace

ws = Workspace.create(name="sql_nube",
                      subscription_id = azure_id,
                      resource_group = "modelo",
                      location = "centralindia")

UserErrorException: UserErrorException:
	Message: You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 72732ed9-d260-4063-9daf-beb8eb11bdad subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscription_id='90eaa2d9-15dc-4bc3-9c4f-2c580923e179')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 72732ed9-d260-4063-9daf-beb8eb11bdad subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscript

In [ ]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

In [11]:
import json
import joblib
import pandas as pd
from azureml.core.model import Model
from datetime import datetime

def init():
    global model, feature_columns
    # Cargar el modelo entrenado y la lista de columnas de características
    model_path = Model.get_model_path('model')
    model = joblib.load(model_path)
    # Asume que feature_columns.pkl fue guardado durante el entrenamiento
    cols_path = Model.get_model_path('feature_columns.pkl')
    feature_columns = joblib.load(cols_path)

def run(raw_data):
    try:
        # Parsear el input JSON y convertir a DataFrame
        input_data = json.loads(raw_data)
        df = pd.DataFrame(input_data['data'])

        # Eliminar columnas que no se usan en scoring
        drop_cols = [
            'CustomerID', 'rowguid', 'ModifiedDate',
            'PasswordHash', 'PasswordSalt'
        ]
        df = df.drop(columns=[c for c in drop_cols if c in df.columns])

        # One-hot encode de las variables categóricas
        df_encoded = pd.get_dummies(df, drop_first=True)

        # Alinear con las columnas de entrenamiento
        df_encoded = df_encoded.reindex(columns=feature_columns, fill_value=0)

        # Generar predicciones (números ordinales) y convertir a fechas
        preds = model.predict(df_encoded)
        dates = [datetime.fromordinal(int(p)).isoformat() for p in preds]

        # Devolver las fechas previstas en formato ISO
        return json.dumps({'predicted_dates': dates})
    except Exception as e:
        return json.dumps({'error': str(e)})


In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn','numpy','joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )